In [6]:
import os
import json
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm
import csv


def crop_from_mask(image, mask):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # If no contours found, return original image
    if not contours:
        return image

    # Find the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(contours[0])

    # Crop the image using the bounding box
    cropped_image = image[y : y + h, x : x + w]
    cropped_mask = mask[y : y + h, x : x + w]

    return cropped_image, cropped_mask


annotation_path = "/data/noah/dataset/ad_human/anno"
out_mask_path = "/data/noah/inference/magna/masks"
out_image_path = "/data/noah/inference/magna/images"
out_csv_path = "/data/noah/inference/magna/instance_infos.csv"
cnt = 0
target_class = "pedestrian"

instance_infos = []

for anno_name in tqdm(os.listdir(annotation_path)):
    anno_path = os.path.join(annotation_path, anno_name)

    with open(anno_path, "r") as f:
        annotation = json.load(f)

    image_path = os.path.join(annotation["parent_path"], annotation["filename"])
    image = Image.open(image_path).convert("RGB")

    # generate crop mask and image
    target_idxs = []
    for idx, anno in enumerate(annotation["annotations"]):
        if (
            anno["label"] == target_class
            and anno["attributes"]["occlusion"] == "0"
            and anno["attributes"]["truncation"] == "0"
        ):
            target_idxs.append(idx)

    for target_idx in target_idxs:
        mask = np.zeros((image.height, image.width))
        points = np.array(annotation["annotations"][target_idx]["points"], dtype=np.int32)
        try:
            mask = cv2.fillPoly(mask, [points], color=255)
        except:
            continue

        cnt += 1
        crop_image, crop_mask = crop_from_mask(np.array(image), mask.astype("uint8"))
        # display(Image.fromarray(crop_image))
        # display(Image.fromarray(crop_mask))
        crop_image = Image.fromarray(crop_image)
        crop_mask = Image.fromarray(crop_mask)

        crop_image.save(os.path.join(out_image_path, "{}.png".format(cnt)))
        crop_mask.save(os.path.join(out_mask_path, "{}.png".format(cnt)))
        info = {
            "image_path": image_path,
            "image_height": crop_mask.height,
        }
        instance_infos.append(info)

sorted_infos = sorted(instance_infos, key=lambda x: x["image_height"])

with open(out_csv_path, mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["image_path", "image_height"])
    writer.writeheader()
    for row in sorted_infos:
        writer.writerow(row)

print("done")

  0%|          | 0/2632 [00:00<?, ?it/s]

100%|██████████| 2632/2632 [05:46<00:00,  7.59it/s]

done


In [3]:
import os
import random
import json
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm


def random_coordinate(mask):
    # 값이 255인 좌표 찾기
    coordinates = np.argwhere(mask == 255)

    # 랜덤으로 좌표 선택
    random_index = random.randint(0, len(coordinates) - 1)
    random_coordinate = coordinates[random_index]

    return tuple(random_coordinate)


def add_mask(mask, new_mask, right, bottom):
    # 새로운 마스크를 더할 위치 계산
    left = right - new_mask.shape[1]
    top = bottom - new_mask.shape[0]

    # 마스크 영역에 새로운 마스크 더하기
    if left < 0 or top < 0:
        return None

    mask[top:bottom, left:right] += new_mask

    return mask


# annotation을 읽고, 특정 영역을 전달 받았을 때 해당 영역내에서 center point 지정.
mask_path = "/data/noah/inference/magna/masks"
mask_lists = [os.path.join(mask_path, mask_name) for mask_name in os.listdir(mask_path)]
target_annotation_path = "/data/noah/dataset/magna_traffic_light/pre_anno"
target_class_name = "sidewalk"
target_height = None
gen_mask_path = "/data/noah/inference/magna/generated_masks"

for ann_idx, ann_name in tqdm(enumerate(os.listdir(target_annotation_path))):
    annotation_path = os.path.join(target_annotation_path, ann_name)

    with open(annotation_path, "r") as f:
        annotation = json.load(f)

    target_index = None

    for idx, ann in enumerate(annotation["annotations"]):
        if ann["label"] == target_class_name:
            target_index = idx
            target_points = ann["points"]
            break

    if target_index is None:
        continue

    # polygon to mask
    mask = np.zeros((annotation["metadata"]["height"], annotation["metadata"]["width"]))
    points = np.array(annotation["annotations"][target_index]["points"], dtype=np.int32)

    while True:
        try:
            target_mask = cv2.fillPoly(np.copy(mask), [points], color=255).astype(np.uint8)
        except:
            break

        rb_spot = random_coordinate(target_mask)  # height, width 순

        # rb_spot x값을 기준으로 height 선정 및 target_height 산출
        target_height = random.randint(100, 400)

        # 사전에 저장한 mask로부터 랜덤으로 하나 선택 후 target_height로 리사이징
        mask_pth = random.choice(mask_lists)
        paste_mask = Image.open(mask_pth)

        ratio = float(target_height) / paste_mask.height
        paste_mask = paste_mask.resize((int(paste_mask.width * ratio), int(paste_mask.height * ratio)))

        paste_mask = np.array(paste_mask)

        mask = add_mask(mask, paste_mask, rb_spot[1], rb_spot[0])

        if mask is None:
            print("None")
            continue
        else:
            mask = Image.fromarray(mask.astype("uint8")).convert("L")
            mask.save(os.path.join(gen_mask_path, "{}.png".format(ann_idx)))
            break

0it [00:00, ?it/s]

18it [00:00, 34.04it/s]

None
None


59it [00:01, 29.34it/s]

None


...

82it [00:02, 33.60it/s]

None
None


90it [00:02, 32.61it/s]

None
None


100it [00:03, 32.20it/s]
